In [86]:
import pandas as pd
import json
import requests

# Step 1: Fetch the JSON data from the URL
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"  # Replace with your actual JSON URL
response = requests.get(url)
with open('restaurant_data.json', 'w') as json_file:  
    json.dump(response.json(), json_file, indent=4) 


In [88]:
def preprocess_restaurant_data():
    with open("restaurant_data.json") as res:
        data = json.load(res)

    restaurant_ids = []
    restaurant_names = []
    restaurant_countries = []
    restaurant_cities = []
    restaurant_votes = []
    restaurant_user_ratings = []
    restaurant_cuisines = []

    for restaurant in data[0]["restaurants"]:
        restaurant_ids.append(restaurant["restaurant"]["R"]["res_id"])
        restaurant_names.append(restaurant["restaurant"]["name"])
        restaurant_countries.append(restaurant["restaurant"]["location"]["country_id"])
        restaurant_cities.append(restaurant["restaurant"]["location"]["city"])
        restaurant_votes.append(restaurant["restaurant"]["user_rating"]["votes"])
        restaurant_user_ratings.append(restaurant["restaurant"]["user_rating"]["aggregate_rating"])
        restaurant_cuisines.append(restaurant["restaurant"]["cuisines"])



    data = {"Restaurant Id" : restaurant_ids, 
            "Restaurant Name" : restaurant_names, 
            "Country Code" : restaurant_countries,
            "City" : restaurant_cities,
            "User Rating Votes" : restaurant_votes,
            "User Aggregate Rating" : restaurant_user_ratings,
            "Cuisines" : restaurant_cuisines
    }

    df = pd.DataFrame(data)
    df.to_csv("restaurant.csv", index=False)



